<a href="https://colab.research.google.com/github/nissimlab/scRNA-seq-fixed-pancreas/blob/main/SupplementaryTable4/CIP_all_immune_fine_DEGs_annotated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Loading Packages

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install scanpy anndata leidenalg watermark pandas==1.5.3 rpy2==3.4.5 umap-learn==0.5.3 plotnine==0.8.0 anndata panel anndata2ri

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 20.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 96.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 72.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
import scanpy as sc
import numpy as np
import re
import os
import pandas as pd
import seaborn as sb
import matplotlib
import scipy.sparse as sp
from scipy.sparse import issparse
import anndata2ri

import matplotlib as mpl
from scipy import stats as scistats
import matplotlib.pyplot as pl
import scanpy.external as sce
from matplotlib import colors
from pathlib import Path
from ipywidgets import interactive

sc._settings.ScanpyConfig.n_jobs=8

pl.rcParams['pdf.fonttype'] = 'truetype'
sc.set_figure_params(vector_friendly=False,dpi_save=300,transparent=True)
pl.rcParams['lines.linewidth'] = 0.1
sc.set_figure_params(color_map='viridis')
colorsComb = np.vstack([pl.cm.Reds(np.linspace(0, 1, 10)), pl.cm.Greys_r(np.linspace(0.7, 0.8, 0))])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)


%matplotlib inline
import panel as pn
pn.extension( comms='colab')

#Setup WD

In [ ]:
folder='/content/drive/MyDrive/CIP_v3_data/CIP_all/Immune/'

samplename='CIP_all_v3_Immune'
random_state=42

import os
os.chdir(folder)

#load in immune from subset analysis

In [ ]:
adata2=sc.read('CIP_all_v3__immune_20230214_annotated.h5ad')

In [ ]:
adata2

AnnData object with n_obs × n_vars = 28073 × 19752
    obs: 'sample', 'donor', 'date', 'n_counts', 'log_counts', 'n_genes', 'mt_frac', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mito', 'log1p_total_counts_mito', 'pct_counts_mito', 'scDblFinder_class', 'doublet_score', 'predicted_doublet', 'batch', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'QC', 'outlier', 'S_score', 'G2M_score', 'phase', 'leiden_r0.8', 'leiden_r0.5', 'leiden_r0.4', 'ADM', 'Acinar', 'Ductal', 'annotated', 'annotated_fine', 'annotated_ADM', 'annotated_immunegrouped', 'leiden_r0.1', 'leiden_r0.2', 'leiden_r0.3', 'leidenX', 'leidenY', 'annotated2', 'leidenZ'
    var: 'feature_type', 'id', 'mito', 'n_cells', 'mt', 'rb', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'tr

In [ ]:
#method : {‘logreg’, ‘t-test’, ‘wilcoxon’, ‘t-test_overestim_var’} | None (default: None)
sc.tl.rank_genes_groups(adata2, "annotated", method='wilcoxon',n_genes=adata2.shape[1],layer='PFlog1pPF_normalization', rankby_abs=False,corr_method='bonferroni',key_added='wilcoxon_rank_genes_groups' )
wilcoxon=pd.DataFrame(adata2.uns['wilcoxon_rank_genes_groups']['names']).head(100)
wilcoxon

,Pancreas resident macrophage,Retnla+ Lyz1+ macrophage,Arg1+ macrophage,Trem2+ Ms4a7+ macrophage,mregDC,cDC1,Cycling cDC1,cDC2,Neutrophils,B,T / NK,Cycling macrophage,Plasmacytoid DC
0,Gm42418,Retnla,Thbs1,Trem2,Tmem123,Plbd1,Top2a,H2-Ab1,Csf3r,Ebf1,Skap1,Mki67,Siglech
1,C1qa,Ear2,Fn1,Ms4a7,Ccr7,Wdfy4,Hmgb2,H2-Eb1,S100a8,Cd79a,Itk,Stmn1,Runx2
2,C1qc,Fn1,Hif1a,Ctsb,Tbc1d8,Naaa,Stmn1,Cd74,S100a9,Bank1,Ets1,Birc5,Mctp2
3,Mrc1,Crip1,Arg1,Ctsd,Mreg,Irf8,Mki67,H2-Aa,Srgn,Igkc,Vps37b,Top2a,Rnase6
4,C1qb,Lyz1,Pkm,Lgmn,Samsn1,Cst3,Pclaf,H2-DMb1,Sorl1,Ighm,Rpl12,Pclaf,Ccr9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Clec10a,Ctrb1,Magt1,C1qa,Rab21,Rpl32,Cenpm,Gpr132,Dock2,Apobec3,Fam107b,Hjurp,Bloc1s2
96,Kctd12,Psap,Nus1,Gas6,Sinhcaf,H2afy,Lrrk2,Ccdc88a,Arhgap15,mt-Co1,Lck,Ckap2,Dntt
97,Atf3,Arhgap39,Glrx,Glul,Atox1,Srgap3,Cit,Rpl37,Ncf2,Shisa5,Fyn,Cep128,Serp1
98,Ext1,Prcp,Timp1,Sec14l1,Serpinb9,Pfkp,Ccser1,Slamf9,Gnai2,Rpl35a,Il2rb,Asf1b,Atp2a1


#DEG table

In [ ]:
make_deg_table(adata2.uns['wilcoxon_rank_genes_groups'], filename= 'immune_deg_table_test2.csv')